In [2]:
%load_ext autoreload
%autoreload 2

import time
from functools import partial

import ipyvuetify as v
import xlwings as xw
import ipywidgets as w
import pandas as pd

from interface import interface

#open excel
wb=xw.Book("IMS.xlsm")

#Instruments
type_of_instruments = ['Equity', 'Bond', 'Fund']
buy_sell = ["Buy","Sell"]


instruments = v.Select(v_model='Equity',id="instrument_select",
    label='Instrument',
    items=type_of_instruments)

def select_change(widget, event, data):
    content_list = [v.TabItem(children=[forms[(instruments.v_model,'Buy')]]),v.TabItem(children=[forms[(instruments.v_model,'Sell')]])]
    tabs = v.Tabs(
    v_model=1, 
    children=tab_list + content_list, color ="blue-grey darken-1")
    content.children = [instruments,tabs,Proceed]
    #compile_excel(wb.sheets["Pre-Check BUY"],cells[(instruments.v_model, 'Buy')])
    

instruments.on_event('change',select_change)

widgets = {}
cells = {}
from copy import deepcopy
for item in interface:
    item = deepcopy(item)
    instrument, type_, obj = item.pop('INSTRUMENT'), item.pop('TYPE'), item.pop('OBJECT')
    widgets.setdefault((instrument, type_), [])
    cells.setdefault((instrument, type_), [])
    label = item['label'].replace(' ','_')
    widget = getattr(v, obj)(**item)
    widget.id = f"{instrument}_{type_}_{label}"
    widgets[instrument, type_].append(widget)
    input_field = True if 'disabled' in item.keys() and  item['disabled']== False  else False
    cells[instrument, type_].append((label,item['cell'],input_field))


#forms    
forms = {}
for k, children in widgets.items():
        forms[k] = v.Form(children=children)

class AllForms:
    def __init__(self):
        for k, form in forms.items():
            for widget in form.children:
                setattr(self, widget.id, widget)

af = AllForms()

#Buttons Form
Proceed = v.Btn(
                  icon = False,
                  children=[
                      v.Icon(children=['mdi-application-export'],  class_ = "mr-2"),
                      'Proceed',
                      ])

def compile_excel(widget,event,data,sht,cells,flg_buy):
    for i,f in enumerate(cells):
        #campi input
        if f[2] == True:  
            obj = getattr(af,f"{instruments.v_model}_{flg_buy}_{f[0]}")
            sht.range(f"{f[1]}").value = obj.v_model 
        #campi output
        else:
            if sht.range(f[1]).value == "#N/A Requesting Data...":
                time.sleep(0.1)
                compile_excel(sht,cells[i:len(cells)])
            else:
                obj = getattr(af,f"{instruments.v_model}_{flg_buy}_{f[0]}")
                obj.v_model = sht.range(f"{f[1]}").value
        

#functions to be called
def Test(sheet_sell):
    sheet_sell.range("C19").value= ticker.v_model  
    if sheet_sell.range("C27").value == "#N/A Requesting Data..." or sheet_sell.range("C30").value == "#N/A Requesting Data...":
        time.sleep(0.1)
        Test(sheet_sell)
    else:
        #print(sheet_sell.range("C27").value)
        Px_Last.v_model = sheet_sell.range("C27").value
        ISIN.v_model = sheet_sell.range("B20").value
        Name.v_model = sheet_sell.range("C20").value
        Crncy.v_model = sheet_sell.range("C28").value
        Cross.v_model = sheet_sell.range("C29").value
        App_Cross.v_model = sheet_sell.range("C30").value
        Price_Euro.v_model = sheet_sell.range("C31").value
        #print(st.range("A5").value)

    #ticker.v_model = sheet_sell.range("C25").value

    
def proceed_event(widget,event,data):
    flg_buy = 'Buy' if tabs.v_model == 0 else 'Sell'
    sheet = wb.sheets['Pre-Check BUY'] if tabs.v_model == 0 else  wb.sheets['Pre-check SELL']
    compile_excel_event = partial(compile_excel, sht = sheet, cells = cells[(instruments.v_model, flg_buy)], flg_buy = flg_buy)
    compile_excel_event(widget,event,data)
    wb.macro("preTradeEquity")
    
Proceed.on_event('click',proceed_event)

   
#form= v.Form(children=[ticker,widgets.HBox([ISIN, Name],style_='width: 100%; opacity: 0.7'),
#                       Px_Last,Crncy,Cross,App_Cross,Price_Euro,Quantity,Verify, Proceed])


#tabs
content_list = [v.TabItem(children=[forms[('Equity','Buy')]]),v.TabItem(children=[forms[('Equity','Sell')]])]
tab_list = [v.Tab(children=["Pre-Trade Buy"]), v.Tab(children=["Pre-Check Sell"])]

tabs = v.Tabs(
    v_model=1, 
    children=tab_list + content_list, color ="blue-grey darken-1")

###############
#APP
###############

#navigation bar
navDrawer = v.NavigationDrawer(v_model=True, children=[
     v.Select(label='select1', items=[1,2,], prepend_icon = 'fa-truck'), 
    v.TextField(label='text', prepend_icon = 'mdi-heart'),
    v.Select(label='select3', prepend_icon = 'mdi-magnify')
])

navDrawer.mini_variant = True
navDrawer.expand_on_hover = True

#navDrawer.width =300
navDrawer.mini_variant_width = 30
navDrawer.mini_variant_width = 0

toolBarButton = v.Btn(
                  icon = True, 
                  children=[
                      v.Icon(children=['mdi-dots-vertical'])
                      ])

def on_click(widget, event, data):
    navDrawer.v_model = not navDrawer.v_model
    if navDrawer.v_model:
        navDrawer.mini_variant_width = 30
    else:
        navDrawer.mini_variant_width = 0
        
#toolBarButton.on_event('click', on_click)

#Init
v.theme.dark = True
app = v.App(v_model=1)

appBar = v.AppBar(
      #color="light-blue darken-3",
    color="blue-grey darken-1",
      dense=True,
      dark = True   ,  
       children = [
           toolBarButton, 
           v.ToolbarTitle(children=['Niche IMS']),
                    ])

content = v.Col(children = [instruments,tabs,Proceed])


drawersWithContent = v.Container(
        class_="fill-height",
         fluid=True,
        children = [
                     v.Row(
                          align="top",
                      justify="left",
                        children = [navDrawer,content]
                            )
                ])

app.children = [appBar,drawersWithContent]


app

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


App(children=[AppBar(children=[Btn(children=[Icon(children=['mdi-dots-vertical'])], icon=True), ToolbarTitle(c…